In this notebook, I run one step for TwoStreamControlModel to get intermediate outputs which I can then compare to my local run

___

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import scripts.control_utils as cu
import torch
torch.set_printoptions(linewidth=200)

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [3]:
torch.use_deterministic_algorithms(True)

In [4]:
path_to_config = 'cnxs_config/sdxl/sdxl_encD_canny_48m.yaml'

If this results in the kernel crashing, I'm using too much GPU memory elsewhere. Shut down every other kernel and try again.

In [5]:
model = cu.create_model(path_to_config).to('cuda')

Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 320, out-chn: 320, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 640, out-chn: 640, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing

In [6]:
size = 768
num_samples=1
prompt='cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed'
n_prompt='lowres, bad anatomy, worst quality, low quality'

In [7]:
def get_canny_edges():
    image_path = 'input_images/shoe.png' # chosen to fit size above
    image = cu.get_image(image_path, size=size)
    edges = cu.get_canny_edges(image, low_th=100, high_th=250)
    return edges
edges = get_canny_edges()

In [8]:
from sgm.umer_debug_logger import udl

In [9]:
udl.set_dir('logs/cloud/', clear=True)
udl.set_condition('SUBBLOCK')

In [10]:
samples, controls, latents = cu.get_sdxl_sample(
    guidance=edges,
    ddim_steps=50,
    num_samples=num_samples,
    model=model,
    shape=[4, size // 8, size // 8],
    control_scale=0.95,
    prompt=prompt,
    n_prompt=n_prompt,
    return_latents=True
)

Global seed set to 1999158951


[CONTROL CORRECTION OF ControlledDiffusionEngine SCALED WITH 0.95]
crop_coords_top_left = tensor([[0, 0]], device='cuda:0')
original_size_as_tuple = tensor([[768, 768]], device='cuda:0')
target_size_as_tuple = tensor([[768, 768]], device='cuda:0')
These are the timesteps: (0: 14.614641189575195)	(1: 12.966320991516113)	(2: 11.542771339416504)	(3: 10.309401512145996)	(4: 9.237422943115234)	(5: 8.302803039550781)	(6: 7.485421180725098)	(7: 6.768382549285889)	(8: 6.137460708618164)	(9: 5.580644607543945)	(10: 5.08776330947876)	(11: 4.650183200836182)	(12: 4.260556697845459)	(13: 3.9126131534576416)	(14: 3.6009886264801025)	(15: 3.3210830688476562)	(16: 3.0689430236816406)	(17: 2.8411612510681152)	(18: 2.634795665740967)	(19: 2.4472994804382324)	(20: 2.276463031768799)	(21: 2.1203653812408447)	(22: 1.977332353591919)	(23: 1.8459018468856812)	(24: 1.7247940301895142)	(25: 1.6128861904144287)	(26: 1.5091912746429443)	(27: 1.4128390550613403)	(28: 1.3230608701705933)	(29: 1.2391756772994995)	

TypeError: TimestepEmbedSequential.forward() missing 1 required positional argument: 'emb'

In [ ]:
sigmas_str = '\t'.join([f'({i}: {t})' for i,t in enumerate([1,2,3])])

In [ ]:
type(model.model.diffusion_model.input_blocks[3][0])

In [ ]:
type(model.model.control_model.input_blocks[3][0])

In [ ]:
from PIL import Image
def show_image(im_tensor): return Image.fromarray(cu.create_image_grid(im_tensor))

In [ ]:
show_image(samples)

In [ ]:
import einops
import numpy as np
from PIL import Image

def lat2img(lat, resize_to=None, output_type='pil'):
    assert lat.dim() == 4, "Expected a batch of images, not a single batch"
    
    with torch.no_grad():
        ims = model.decode_first_stage(lat)
        ims = (einops.rearrange(
            ims, 'b c h w -> b h w c'
        ) * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)
        
        if output_type == 'pil': ims = [Image.fromarray(im) for im in ims]
        
        if resize_to is not None:
            if output_type=='pil': ims = [im.resize(resize_to) for im in ims]
            else: print(f'Not resizing as output_type = {output_type} requested')
    return ims

In [ ]:
from functools import partial
from tqdm.notebook import tqdm

import torch
import einops
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw
from tqdm.notebook import tqdm
from functools import partial

def plot_latents_to_pil_grid(lats, every=1, cols=10, im_size=200, pbar=True, border=2, return_ims=True, output_type='pil'):
    if not isinstance(im_size, (list, tuple)): im_size = (im_size, im_size)
    
    lats = [lat for i, lat in enumerate(lats) if i % every == 0 or i == len(lats)-1]
    if pbar: lats = tqdm(lats)
    
    # decoce latents -> images
    ims = [lat2img(lat, resize_to=im_size, output_type=output_type)[0] for lat in lats] # removed pipe argument
    
    # add border
    ims_bordered = [ImageOps.expand(im, border=2, fill='black') for im in ims]
    im_size = (im_size[0]+border, im_size[1]+border)

    rows = len(ims) // cols
    if rows * cols < len(ims): rows += 1

    # draw background
    grid_image = Image.new('RGB', (cols * im_size[0], rows * im_size[1]), color='grey')
    draw = ImageDraw.Draw(grid_image)
    for xy in range(0,2*max(cols * im_size[0], rows * im_size[1])+1,100):
        draw.line([(xy, 0), (0, xy)], fill="white", width=1)
    
    # draw images
    for i, img in enumerate(ims_bordered):
        x_offset = (i % cols) * im_size[0]
        y_offset = (i // cols) * im_size[1]
        grid_image.paste(img, (x_offset, y_offset))

    if return_ims: return grid_image, ims
    else: return grid_image

In [ ]:
grid, ims = plot_latents_to_pil_grid(latents)

Cuda Heidelberg, control scale = 0

In [ ]:
grid